In [ ]:
!python --version

In [ ]:
import sys

print(sys.executable)

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
ten = torch.exp(torch.tensor(0.03))
print(ten)
ten.item()
print(type(ten.item()))

In [ ]:
sst2 = pd.read_csv("sst2_dataset.txt", sep="\t")

In [ ]:
sst2.head(3)

In [ ]:
len(sst2)

In [ ]:
sst2.iloc[2]["sentence"]

In [ ]:
wikitext = pd.read_csv("wiki.valid.raw", sep="\t")

In [ ]:
wikitext.head(3)

In [ ]:
len(wikitext)

In [ ]:
sst2 = sst2["sentence"]
sst2.head(3)

In [ ]:
sst2.to_csv("sst2_preprocessed.txt", index=False, header=False)

In [ ]:
sst2.iloc[0]

In [ ]:
msk = np.random.rand(len(sst2)) < 0.8
train = sst2[msk]
test = sst2[~msk]
print(len(train))
train.head(2)

In [ ]:
print(len(test))

In [ ]:
train.to_csv("sst2_train.txt", index=False, header=False)
test.to_csv("sst2_test.txt", index=False, header=False)

In [ ]:
plm = GPT2LMHeadModel.from_pretrained("models/" + model_name, pad_token_id = tokenizer.eos_token_id)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DistilBertForMaskedLM


# model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
model = DistilBertForMaskedLM.from_pretrained(
    "gpt2_fine_tuned_sst2",
    local_files_only=True
)

In [ ]:
model

In [ ]:
device = torch.device("cuda")
model = model.to(device)

In [ ]:
model.is_cuda

In [ ]:
torch.cuda.is_available()

In [ ]:
# import polyjuice_nlp

In [1]:
import polyjuice

In [2]:
# initiate a wrapper.
# model path is defaulted to our portable model:
# https://huggingface.co/uw-hai/polyjuice
# No need to change this unless you are using customized model
pj = polyjuice.Polyjuice(model_path="uw-hai/polyjuice", is_cuda=True)

In [3]:
# the base sentence
text = "It is great for kids."
perturbations = pj.perturb(orig_sent=text)

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
INFO:polyjuice.polyjuice_wrapper:Setup perplexity scorer.


In [6]:
# perturb the sentence with one line:
# When running it for the first time, the wrapper will automatically
# load related models, e.g. the generator and the perplexity filter.
perturbations = pj.perturb(
    orig_sent=text,
    # can specify where to put the blank. Otherwise, it's automatically selected.
    # Can be a list or a single sentence.
    blanked_sent="It is [BLANK] for kids.",
    # can also specify the ctrl code (a list or a single code.)
    # The code should be from 'resemantic', 'restructure', 'negation', 'insert', 'lexical', 'shuffle', 'quantifier', 'delete'.
    ctrl_code="negation",
    # Customzie perplexity score.
    perplex_thred=5,
    # number of perturbations to return
    num_perturbations=1,
    # the function also takes in additional arguments for huggingface generators.
    num_beams=3
)

INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

is_cuda = True
model_path = "uw-hai/polyjuice"
generator = pipeline("text-generation",
                     model=AutoModelForCausalLM.from_pretrained(model_path),
                     max_length=2048,
                     tokenizer=AutoTokenizer.from_pretrained(model_path),
                     framework="pt", device=0 if is_cuda else -1)

prompt_text = "A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman."
generator(prompt_text, num_beams=3, num_return_sequences=3)

In [ ]:
# prompt_text = "A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman."

In [ ]:
counter_review = "Because IT IS, that's why! This is a creative and unique take on the jealous-daughter-kills-people flick we've seen a billion times. Rosanna Arquette makes anything worth watching, and Mandy Schaffer's brief nude scene (after teasing via scantily clad attire throughout the film) at the end is yet another reason to watch this fantastic film **** out of ****	1"

In [ ]:
review_prompt = "Because IT IS, that's why! This is the same jealous-daughter-kills-people flick we've seen a billion times. Rosanna Arquette makes anything worth watching, and Mandy Schaffer's brief nude scene (after teasing via scantily clad attire throughout the film) at the end almost make this trite blarney worthwhile, but not quite.* out of **** 0 <|perturb|> [BLANK] [SEP]"

# review_prompt = "Because IT IS, that's why! This is the same jealous-daughter-kills-people flick we've seen a billion times. Rosanna Arquette makes anything worth watching, and Mandy Schaffer's brief nude scene (after teasing via scantily clad attire throughout the film) at the end almost make this trite blarney worthwhile, but not quite.* out of **** 0 <|perturb|> [negation] Because IT IS, that's why! This is the same jealous-daughter-kills-people flick we've seen a billion times. Rosanna Arquette [BLANK], and Mandy Schaffer's brief nude scene (after teasing via scantily clad attire throughout the film) at the end almost make this trite blarney worthwhile, but not quite.* out of **** 0 [SEP]"

# short_review_prompt = "This is the same jealous-daughter-kills-people flick we've seen a billion times. <|perturb|> [negation] [BLANK]"

In [ ]:
generator(review_prompt, num_beams=3, num_return_sequences=3)
# generator(short_review_prompt, num_beams=3, num_return_sequences=3)

In [ ]:
from polyjuice.generations import ALL_CTRL_CODES

In [ ]:
# short_output #[0]['generated_text']

In [ ]:
prompt_text = "A dog is embraced by the woman. <|perturb|> [negation] [BLANK]"
generator(prompt_text, num_beams=3, num_return_sequences=3)

In [ ]:
# dksl = "There's nothing wrong with a popcorn movie to keep you off the streets. It's just that some are better than others. This is very excellent. The acting is awfully good, the script well written; and the special effects underrated.Why does Hollywood treat it's audience with such intrigue? And why have they not made a sequel?	1	There's nothing wrong with a popcorn movie to keep you off the streets. It's just that some are better than others. This is very poor. The acting is awful, the script dire; and the special effects overrated.Why does Hollywood treat it's audience with such contempt? And why have they made a sequel?"

short_review_prompt = "There's nothing wrong with a popcorn movie to keep you off the streets."
generator(short_review_prompt, num_beams=3, num_return_sequences=3)

In [59]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# initialize tokenizer and model from pretrained GPT2 model
# tokenizer = GPT2Tokenizer.from_pretrained('sshleifer/tiny-gpt2')
# model = GPT2LMHeadModel.from_pretrained('sshleifer/tiny-gpt2')

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [62]:
sequence = "Worst movie, (with the best reviews given it) I've ever seen. Over the top dialog, acting, and direction. more slasher flick than thriller.With all the great reviews this movie got I'm appalled that it turned out so silly."
inputs = tokenizer.encode(sequence, return_tensors='pt')
outputs = model.generate(inputs, max_length=120,
                         do_sample=True,
                         no_repeat_ngram_size=0,
                         num_beam=5,
                         temperature=0.6,
                         num_return_sequences=1)
out_1 = tokenizer.decode(outputs[0], skip_special_tokens=True)

outputs = model.generate(inputs, max_length=120,
                         do_sample=True,
                         no_repeat_ngram_size=0,
                         num_beam=5,
                         temperature=0.6,
                         num_return_sequences=1)
out_2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Is out1 == out2?? {out_1 == out_2}\n")
print(out_1)
print("\n\n\n")
print(out_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Is out1 == out2?? False

Worst movie, (with the best reviews given it) I've ever seen. Over the top dialog, acting, and direction. more slasher flick than thriller.With all the great reviews this movie got I'm appalled that it turned out so silly. I'm not sure why. I'm not sure why. I don't know why. I don't know why. I'm not sure why. I don't know why. I don't know why. I'm not sure why. I don't know why. I'm not sure why. I'm not sure why.




Worst movie, (with the best reviews given it) I've ever seen. Over the top dialog, acting, and direction. more slasher flick than thriller.With all the great reviews this movie got I'm appalled that it turned out so silly. But I was also shocked to see that all those big screen reviews were not the same as this film. I couldn't believe that the action was so bad because I didn't know what to expect. I'm not sure what to expect.The film is very action packed and action packed, the characters are just as strong as the characters in


In [11]:
# import ast
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')

In [10]:
# sentences = parser.raw_parse_sents(["Hello, My name is Melroy.", "What is your name?"])

In [16]:
seed_review = nlp('This is a test. But this is the second sentence. And we made more of it.')
for sentence in seed_review.sentences:
    print(sentence.constituency)

(ROOT (S (NP (DT This)) (VP (VBZ is) (NP (DT a) (NN test))) (. .)))
(ROOT (S (CC But) (NP (DT this)) (VP (VBZ is) (NP (DT the) (JJ second) (NN sentence))) (. .)))


In [15]:
counter_review = nlp('This is not a test. But this is the second sentence.')
for sentence in counter_review.sentences:
    print(sentence.constituency)

(ROOT (S (NP (DT This)) (VP (VBZ is) (NP (DT a) (NN test))) (. .)))
(ROOT (S (CC But) (NP (DT this)) (VP (VBZ is) (NP (DT the) (JJ second) (NN sentence))) (. .)))


In [58]:
# parse counter review
seed_review = nlp('This is a test. But this is the second sentence. And we made more of it.')

# parse generated counter review
counter_review = nlp('This is not a test. But this is the second sentence. And.')

# check and add empty sentences
if len(seed_review.sentences) >= len(counter_review.sentences):
    n_to_add = len(seed_review.sentences) - len(counter_review.sentences)
    to_add = [nlp(".").sentences[0] for i in range(n_to_add)]
    counter_review.sentences += to_add
else:
    n_to_add = len(counter_review.sentences) - len(seed_review.sentences)
    to_add = [nlp(".").sentences[0] for i in range(n_to_add)]
    seed_review.sentences += to_add

print(len(seed_review.sentences))
print(len(counter_review.sentences))
assert len(seed_review.sentences) == len(counter_review.sentences)

# print to manual check
print(len(seed_review.sentences) == len(counter_review.sentences))
for sentence in seed_review.sentences:
    print(sentence.constituency)

for sentence in counter_review.sentences:
    print(sentence.constituency)

3
3
True
(ROOT (S (NP (DT This)) (VP (VBZ is) (NP (DT a) (NN test))) (. .)))
(ROOT (S (CC But) (NP (DT this)) (VP (VBZ is) (NP (DT the) (JJ second) (NN sentence))) (. .)))
(ROOT (S (CC And) (NP (PRP we)) (VP (VBD made) (NP (NP (JJR more)) (PP (IN of) (NP (PRP it))))) (. .)))
(ROOT (S (NP (DT This)) (VP (VBZ is) (RB not) (NP (DT a) (NN test))) (. .)))
(ROOT (S (CC But) (NP (DT this)) (VP (VBZ is) (NP (DT the) (JJ second) (NN sentence))) (. .)))
(ROOT (S (CC And) (. .)))


In [57]:
type(seed_review)

stanza.models.common.doc.Document

In [51]:
import zss
import numpy as np

distances = [zss.simple_distance(s_one.constituency, s_two.constituency) for (s_one, s_two) in zip(seed_review.sentences, counter_review.sentences)]

np.mean(distances)

7.0

In [56]:
import Levenshtein
Levenshtein.distance("ciao", "ciao")

0

In [4]:
import nltk
smoothing_function = nltk.translate.bleu_score.SmoothingFunction()

In [31]:
reference = "my name is Diego and I am fine but feeling odd sometimes. Yes, please. Amigoooooo. no me lo puedo creer"
hypothesis = "my name is Diego and I am fine but feeling odd sometimes. What about you?"
reference = nltk.tokenize.word_tokenize(reference)
hypothesis = nltk.tokenize.word_tokenize(hypothesis)

nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, smoothing_function=smoothing_function.method1)

0.4904617366562973

In [32]:
nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, smoothing_function=smoothing_function.method2)

0.4988968343630959

In [33]:
import fast_bleu

In [37]:
list_of_references = ["I am Diego", "I am Diego Carraro"]
list_of_references = [nltk.tokenize.word_tokenize(el) for el in list_of_references]
weights = {'bigram': (1/2., 1/2.), 'trigram': (1/3., 1/3., 1/3.), '4_gram': (1/4., 1/4., 1/4., 1/4.)}
self_bleu = fast_bleu.SelfBLEU(list_of_references, weights)
self_bleu.get_score()

{'bigram': [0.7165313105737893, 0.7071067811865476],
 'trigram': [0.7165313105737893, 0.6299605162718741],
 '4_gram': [0.40293516672844226, 0.3976353643835253]}

In [39]:
4.5 / 3

1.5

In [40]:
4.5 // 3

1.0